In [80]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.7 MB/s eta 0:00:00


In [22]:
!python -m spacy download en_core_web_lg -q

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-17 16:12:25.560166: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-17 16:12:25.560304: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-17 16:12:25.560327: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [125]:
import pandas as pd
import string
import re
import numpy as np 
import sklearn
from sklearn.model_selection import train_test_split


# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')

import en_core_web_lg
from spacy.lang.en.stop_words import STOP_WORDS

import tensorflow as tf

from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification



In [177]:
df = pd.read_csv("/content/spam.csv", encoding = "ISO-8859-1")

In [178]:
len(df)

5572

In [25]:
df.head()
# "ham" messages are those that are not spam

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [26]:
print(df.columns)

# Dropping empty columns
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)

# Giving more explicit names to columns
df.rename(columns = {"v1" : "type"}, inplace = True)
df.rename(columns = {"v2" : "text"}, inplace = True)

# We encode the values in column "type": 0 for non-spam messages, 1 for spam
df["type"] = df["type"].apply(lambda x: 0 if x=="ham" else 1)

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')


In [30]:
(df["type"]).value_counts()

ham     4825
spam     747
Name: type, dtype: int64

In [33]:
df['type'].nunique()

2

In [35]:
# checking if there are missing values in the dataset
df.isnull().sum() 

type    0
text    0
dtype: int64

#Text Preprocessing



In [37]:
nlp = en_core_web_lg.load()

In [61]:
# Removing punctuation by keeping only alphanumeric characters in the text
df["text_clean"] = df["text"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))

# Removing capitalization
df["text_clean"] = df["text_clean"].apply(lambda x: x.replace("  "," ").lower().strip())

# Removing stop words
df["text_clean"] = df["text_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))

In [62]:
df.head()

,type,text,text_clean
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun early hor u c
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think usf live


In [63]:
for i in range(0, 15):
  print(df["type"].loc[i], df["text"].loc[i])

0 Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
0 Ok lar... Joking wif u oni...
1 Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
0 U dun say so early hor... U c already then say...
0 Nah I don't think he goes to usf, he lives around here though
1 FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
0 Even my brother is not like to speak with me. They treat me like aids patent.
0 As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
1 WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
1 Had your mobile 11 mont

In [65]:
spam_df = df[df["type"]==1].copy()
spam_df = spam_df.reset_index(drop = True)

In [66]:
ham_df = df[df["type"]==0].copy()
ham_df = ham_df.reset_index(drop = True)

In [67]:
spam_df.head()

,type,text,text_clean
0,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
1,1,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darle 3 week word d like fun tb ok...
2,1,WINNER!! As a valued network customer you have...,winner value network customer select receivea ...
3,1,Had your mobile 11 months or more? U R entitle...,mobile 11 month u r entitle update late colour...
4,1,"SIX chances to win CASH! From 100 to 20,000 po...",chance win cash 100 20000 pound txt csh11 send...


In [68]:
ham_df.head()

,type,text,text_clean
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,0,U dun say so early hor... U c already then say...,u dun early hor u c
3,0,"Nah I don't think he goes to usf, he lives aro...",nah think usf live
4,0,Even my brother is not like to speak with me. ...,brother like speak treat like aids patent


In [166]:
# Checking is some rows of the dataframe became empty after text cleaning:
empty_rows_indices = []
for i in range(0, len(df)):
  if len(df["text_clean"].loc[i].split())==0:
    empty_rows_indices.append(i)

print(empty_rows_indices)

[43, 959, 1087, 1190, 1236, 1407, 2740, 2805, 2871, 2927, 3374, 3767, 4271, 4573, 4640, 4822, 5529]


In [176]:
len(empty_rows_indices)

17

In [167]:
df["text_clean"].loc[43]

''

In [168]:
df["text_clean"].loc[42]

'07732584351 rodger burn msg try reply sm free nokia mobile free camcorder 08000930705 delivery tomorrow'

In [173]:
df.drop(axis=0, index=empty_rows_indices, inplace=True)

In [175]:
len(df)

5555

In [174]:
df["text_clean"].loc[43]

KeyError: ignored

In [ ]:
for index in empty_row_indices:

In [107]:
print(df["text_clean"].loc[0])

jurong point crazy available bugis n great world la e buffet cine amore wat


In [108]:
len(df["text_clean"].loc[0])

75

In [109]:
len(df["text_clean"].loc[0].split())

14

In [116]:
# After cleaning, how many words does the longest message contain?
max_len = 0

for i in range(0, len(df)):
  max_len = max(max_len, len(df["text_clean"].loc[i].split()))

print("Maximum message length in words:", max_len)

Maximum message length in words: 72


In [69]:
for i in range(0, 15):
  print(spam_df["text_clean"].loc[i])

free entry 2 wkly comp win fa cup final tkts 21st 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s
freemsg hey darle 3 week word d like fun tb ok xxx std chgs send å150 rcv
winner value network customer select receivea å900 prize reward claim 09061701461 claim code kl341 valid 12 hour
mobile 11 month u r entitle update late colour mobile camera free mobile update co free 08002986030
chance win cash 100 20000 pound txt csh11 send 87575 cost 150pday 6days 16 tsandcs apply reply hl 4 info
urgent win 1 week free membership å100000 prize jackpot txt word claim 81010 tc wwwdbuknet lccltd pobox 4403ldnw1a7rw18
xxxmobilemovieclub use credit click wap link txt message click httpwap xxxmobilemovieclubcomnqjkgighjjgcbl
england v macedonia miss goalsteam news txt ur national team 87077 eg england 87077 trywale scotland 4txtì¼120 poboxox36504w45wq 16
thank subscription ringtone uk mobile charge å5month confirm reply yes reply charge
07732584351 rodger burn msg try r

In [70]:
for i in range(0, 15):
  print(ham_df["text_clean"].loc[i])

jurong point crazy available bugis n great world la e buffet cine amore wat
ok lar joke wif u oni
u dun early hor u c
nah think usf live
brother like speak treat like aids patent
request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune
m home soon want talk stuff anymore tonight k ve cry today
ve search right word thank breather promise help grant fulfil promise wonderful blessing time
date sunday
oh kim watch
eh u remember 2 spell yes v naughty v wet
fine thatåõs way u feel thatåõs way gota b
seriously spell
iûm try 2 month ha ha joke
ì pay lar da stock comin


In [121]:
# Saving text of sms messages as numpy array
texts = df["text_clean"].values
print(X[:5])

# Saving labels as numpy array
labels = df["type"].values
print(y[:5])

['jurong point crazy available bugis n great world la e buffet cine amore wat'
 'ok lar joke wif u oni'
 'free entry 2 wkly comp win fa cup final tkts 21st 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s'
 'u dun early hor u c' 'nah think usf live']
[0 0 1 0 0]


In [122]:
texts.shape

(5572,)

In [123]:
labels.shape

(5572,)

In [124]:
# Splitting the arrays into training and validation datasets
texts_train, texts_val, labels_train, labels_val = train_test_split(texts, labels)

In [126]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [127]:
print('Actual text:' , texts[2])

Actual text: free entry 2 wkly comp win fa cup final tkts 21st 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s


In [131]:
print('Tokens:' , tokenizer.tokenize(texts[2]))

Tokens: ['free', 'entry', '2', 'w', '##k', '##ly', 'com', '##p', 'win', 'fa', 'cup', 'final', 't', '##kt', '##s', '21st', '2005', 'text', 'fa', '87', '##12', '##1', 'receive', 'entry', 'questions', '##t', '##d', 'tx', '##t', 'rate', '##tc', '##s', 'apply', '08', '##45', '##28', '##100', '##75', '##over', '##18', '##s']


In [132]:
print('Token to ids:', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[2])))

Token to ids: [2489, 4443, 1016, 1059, 2243, 2135, 4012, 2361, 2663, 6904, 2452, 2345, 1056, 25509, 2015, 7398, 2384, 3793, 6904, 6584, 12521, 2487, 4374, 4443, 3980, 2102, 2094, 19067, 2102, 3446, 13535, 2015, 6611, 5511, 19961, 22407, 18613, 23352, 7840, 15136, 2015]


In [133]:
print(texts[:5])

['jurong point crazy available bugis n great world la e buffet cine amore wat'
 'ok lar joke wif u oni'
 'free entry 2 wkly comp win fa cup final tkts 21st 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s'
 'u dun early hor u c' 'nah think usf live']


In [134]:
type(texts)

numpy.ndarray

In [119]:
encoded_dict = tokenizer.encode_plus(X[0],
                                    add_special_tokens = True,
                                    max_length = max_len,
                                    padding = 'max_length',
                                    return_attention_mask = True)

In [103]:
type(encoded_dict)

transformers.tokenization_utils_base.BatchEncoding

In [120]:
encoded_dict.keys

<bound method BatchEncoding.keys of {'input_ids': [101, 18414, 17583, 2391, 4689, 2800, 11829, 2483, 1050, 2307, 2088, 2474, 1041, 28305, 25022, 2638, 26297, 28194, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}>

In [102]:
print("dict", encoded_dict['input_ids'])

dict [101, 18414, 17583, 2391, 4689, 2800, 11829, 2483, 1050, 2307, 2088, 2474, 1041, 28305, 25022, 2638, 26297, 28194, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Configuring the model

In [140]:
# Creating inputs for BERT model
input_ids = []
attention_masks = []

for text in texts:
  encoded_dict = tokenizer.encode_plus(text,
                                    add_special_tokens = True,
                                    max_length = max_len,
                                    padding = 'max_length',
                                    return_attention_mask = True)
  
  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])

In [141]:
print(input_ids)

[[101, 18414, 17583, 2391, 4689, 2800, 11829, 2483, 1050, 2307, 2088, 2474, 1041, 28305, 25022, 2638, 26297, 28194, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 7929, 2474, 2099, 8257, 15536, 2546, 1057, 2006, 2072, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2489, 4443, 1016, 1059, 2243, 2135, 4012, 2361, 2663, 6904, 2452, 2345, 1056, 25509, 2015, 7398, 2384, 3793, 6904, 6584, 12521, 2487, 4374, 4443, 3980, 2102, 2094, 19067, 2102, 3446, 13535, 2015, 6611, 5511, 19961, 22407, 18613, 23352, 7840, 15136, 2015, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1057, 24654, 2220, 7570, 2099, 1057, 1039, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [158]:
len(input_ids[-1])

72

In [152]:
# Define a nested list
my_nested_list = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Convert the nested list to a tensor
my_tensor = tf.convert_to_tensor(my_nested_list)

# Print the tensor
print(my_tensor)

tf.Tensor(
[[1 2 3]
 [4 5 6]
 [7 8 9]], shape=(3, 3), dtype=int32)


In [153]:
input_ids.convert_to_tensor

AttributeError: ignored

In [154]:
input_ids_tensor = tf.convert_to_tensor(input_ids)

ValueError: ignored

In [142]:
len(input_ids)

5572

In [144]:
len(attention_masks)

5572

In [150]:
my_list = [1, 2, 3]
my_tensor = tf.convert_to_tensor(my_list)

In [151]:
my_tensor

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [146]:
input_ids = tf.convert_to_tensor(input_ids)
attention_masks = tf.convert_to_tensor(attention_masks)

ValueError: ignored

In [99]:
def mask_inputs_for_bert(X, max_len):
  input_ids = []
  attention_masks = []
  i = 0
  for sms in X:
    if (i<3):
      print("sms:", sms)
      
      encoded_dict = tokenizer.encode_plus(sms,
                                       add_special_tokens = True,
                                       max_length = max_len,
                                       pad_to_max_length = True,
                                       return_attention_mask = True)
    if (i<3):
      print("dict", encoded_dict['input_ids'])

      input_ids.append(encoded_dict['input_ids'])

      attention_masks.append(encoded_dict['attention_mask'])

      i = i+1

  input_ids = tf.convert_to_tensor(input_ids)
  attention_masks = tf.convert_to_tensor(attention_masks)
  return input_ids, attention_masks
  


In [19]:
#Configure the model
BERT_MODEL = "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2"
# Importing the preprocessing thant matches the model
PREPROCESS_MODEL = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [20]:
sentences = df["text"]

In [21]:
sentences

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: text, Length: 5572, dtype: object

In [22]:
for i in range(0, 15):
  print(df["text_clean"].loc[i])
  

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
U dun say so early hor... U c already then say...
Nah I don't think he goes to usf, he lives around here though
FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
Even my brother is not like to speak with me. They treat me like aids patent.
As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Had your mobile 11 months or more? U R enti

In [23]:
# we create a copy of the dataframe to look more closely 
spam_df = df[df["type"]=="spam"].copy()
spam_df = spam_df.reset_index(drop = True)

In [24]:
"""for i in range(0, 15):
  print(spam_df["text"].loc[i])
  print()"""

'for i in range(0, 15):\n  print(spam_df["text"].loc[i])\n  print()'

In [25]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df["text_clean"])
df["text_encoded"] = tokenizer.texts_to_sequences(df["text_clean"])

In [26]:
df.head()

,type,text,text_clean,text_encoded
0,0,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ...","[50, 469, 4410, 841, 751, 657, 64, 8, 1324, 89..."
1,0,Ok lar... Joking wif u oni...,Ok lar... Joking wif u oni...,"[46, 336, 1495, 470, 6, 1929]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[47, 486, 8, 19, 4, 796, 899, 2, 178, 1930, 11..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...,"[6, 245, 152, 23, 379, 2989, 6, 140, 154, 57, ..."
4,0,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro...","[1018, 1, 98, 107, 69, 487, 2, 956, 69, 1933, ..."


In [27]:
#df["text_padded_encoded"] = df["text_encoded"].apply(lambda x: tf.keras.preprocessing.sequence.pad_sequences(x, padding="post"))

In [28]:
df.head()

,type,text,text_clean,text_encoded
0,0,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ...","[50, 469, 4410, 841, 751, 657, 64, 8, 1324, 89..."
1,0,Ok lar... Joking wif u oni...,Ok lar... Joking wif u oni...,"[46, 336, 1495, 470, 6, 1929]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[47, 486, 8, 19, 4, 796, 899, 2, 178, 1930, 11..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...,"[6, 245, 152, 23, 379, 2989, 6, 140, 154, 57, ..."
4,0,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro...","[1018, 1, 98, 107, 69, 487, 2, 956, 69, 1933, ..."


In [29]:
text_pad = tf.keras.preprocessing.sequence.pad_sequences(df["text_encoded"], padding="post")

In [30]:
full_ds = tf.data.Dataset.from_tensor_slices((text_pad, df["type"]))

In [31]:
df.shape

(5572, 4)

In [32]:
TAKE_SIZE = int(0.7*df.shape[0])

train_data = full_ds.take(TAKE_SIZE).shuffle(TAKE_SIZE)
train_data = train_data.batch(64)

test_data = full_ds.skip(TAKE_SIZE)
test_data = test_data.batch(64)

In [33]:
type(train_data.take(1))

tensorflow.python.data.ops.dataset_ops.TakeDataset

In [34]:
for text, text_type in train_data.take(1):
  print(text, text_type)

tf.Tensor(
[[  70    2   68 ...    0    0    0]
 [4164  645    9 ...    0    0    0]
 [  94  453   13 ...    0    0    0]
 ...
 [4072 1363 1607 ...    0    0    0]
 [ 641   13  883 ...    0    0    0]
 [1970   20  108 ...    0    0    0]], shape=(64, 189), dtype=int32) tf.Tensor(
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0], shape=(64,), dtype=int64)


In [35]:
type(text)

tensorflow.python.framework.ops.EagerTensor

In [36]:
text.shape

TensorShape([64, 189])

In [37]:
text.shape[1]

189

In [38]:
text

<tf.Tensor: shape=(64, 189), dtype=int32, numpy=
array([[  70,    2,   68, ...,    0,    0,    0],
       [4164,  645,    9, ...,    0,    0,    0],
       [  94,  453,   13, ...,    0,    0,    0],
       ...,
       [4072, 1363, 1607, ...,    0,    0,    0],
       [ 641,   13,  883, ...,    0,    0,    0],
       [1970,   20,  108, ...,    0,    0,    0]], dtype=int32)>

In [39]:
df.shape[1]

4

#Transfer Learning

In [ ]:
base_model = 

In [40]:
from tensorflow.keras.layers import Embedding, Dense, LSTM

vocab_size = len(tokenizer.word_index)
model_lstm = tf.keras.Sequential([
                  Embedding(vocab_size+1, 64, input_shape=[text.shape[1],],name="embedding"),
                  # text.shape[1]: 72
                  LSTM(units=64, return_sequences=True, name = "ltsm_1"), # maintains the sequential nature
                  LSTM(units=64, return_sequences=False, name = "lstm_2"), # returns the last output
                  Dense(16, activation='relu', name = "dense_1"),
                  Dense(8, activation='relu'),
                  Dense(1, activation="sigmoid", name="last")
                  ])

'from tensorflow.keras.layers import Embedding, Dense, LSTM\n\nvocab_size = len(tokenizer.word_index)\nmodel_lstm = tf.keras.Sequential([\n                  Embedding(vocab_size+1, 64, input_shape=[text.shape[1],],name="embedding"),\n                  # text.shape[1]: 72\n                  #LSTM(units=64, return_sequences=True, name = "ltsm_1"), # maintains the sequential nature\n                  LSTM(units=64, return_sequences=False, name = "lstm_2"), # returns the last output\n                  Dense(16, activation=\'relu\', name = "dense_1"),\n                  Dense(8, activation=\'relu\'),\n                  Dense(1, activation="sigmoid", name="last")\n                  ])'

In [42]:
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 189, 64)           570944    
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
 last (Dense)                (None, 1)                 17        
                                                                 
Total params: 605,025
Trainable params: 605,025
Non-trainable params: 0
_________________________________________________________________


In [43]:
optimizer= tf.keras.optimizers.Adam()

model_lstm.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [44]:
model_lstm.fit(train_data,
              epochs=100, 
              validation_data=test_data,
               class_weight=weights)

Epoch 1/100
61/61 [==============================] - 33s 482ms/step - loss: 0.6914 - binary_accuracy: 0.8526 - val_loss: 0.6915 - val_binary_accuracy: 0.8636
Epoch 2/100
61/61 [==============================] - 14s 228ms/step - loss: 0.6913 - binary_accuracy: 0.8669 - val_loss: 0.6925 - val_binary_accuracy: 0.8636
Epoch 3/100
61/61 [==============================] - 12s 197ms/step - loss: 0.6913 - binary_accuracy: 0.4462 - val_loss: 0.6945 - val_binary_accuracy: 0.1364
Epoch 4/100
61/61 [==============================] - 13s 212ms/step - loss: 0.6910 - binary_accuracy: 0.1656 - val_loss: 0.6938 - val_binary_accuracy: 0.1364
Epoch 5/100
61/61 [==============================] - 15s 242ms/step - loss: 0.6910 - binary_accuracy: 0.3062 - val_loss: 0.6935 - val_binary_accuracy: 0.1364
Epoch 6/100
61/61 [==============================] - 13s 210ms/step - loss: 0.6910 - binary_accuracy: 0.7792 - val_loss: 0.6918 - val_binary_accuracy: 0.8636
Epoch 7/100
61/61 [==============================] -

In [45]:
model_lstm.save("model_lstm.h5")

In [46]:
import json
json.dump(model_lstm.history.history, open("/content/LSTM_history.json", 'w'))

In [47]:
LSTM_history = json.load(open("/content/LSTM_history.json", 'r'))
model_lstm = tf.keras.models.load_model("/content/model_lstm.h5")

In [48]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=LSTM_history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=LSTM_history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()
